In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import json
import time
import requests
import csv


In [3]:
curr = os.getcwd()

In [4]:
output_directory = "scraped_data"  
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
os.chdir(f'{curr}/scraped_data')

In [5]:
os.getcwd()

'/Users/yings/Desktop/govtech/section2/scraped_data'

In [10]:
## given to scrape for parliament speeches and staements ##
 
driver = webdriver.Chrome()  # Modify 'path_to_chromedriver' accordingly
page_url = 'https://sprs.parl.gov.sg/search/home'
driver.get(page_url)

# Get search box and fill it up
search_selector = '''#divmpscreen2 > div.row > div:nth-child(1) > div > div:nth-child(1) > input'''
search = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, search_selector))
)
search.send_keys('COS')

# Uncomment following lines to only search in titles
# checkbox_selector = '''#divmpscreen2 > div.row > div:nth-child(1) > div > div:nth-child(2) > label > input'''
# checkbox = driver.find_element(By.CSS_SELECTOR, checkbox_selector)
# checkbox.click()

# Select the 13th parliament
session_selector = '''#divmpscreen2 > div.row > div:nth-child(1) > div > div.form-group.byParText > select > option:nth-child(14)'''
session = driver.find_element(By.CSS_SELECTOR, session_selector)
session.click()

# Find submit element and click
time.sleep(2)  

# Now find and click the submit button
submit_selector = '''#divmpscreen2 > div.row > div.col-sm-12.text-right.pull-right > div > button:nth-child(2)'''
submit = driver.find_element(By.CSS_SELECTOR, submit_selector)
driver.execute_script("arguments[0].scrollIntoView(true);", submit)
time.sleep(1)  
submit.click()


print('Search parameters submitted.')

# Create empty dictionary to store results
res_dict = {}

# Switch window and check for number of search results
driver.switch_to.window(driver.window_handles[1])
num_results_selector = '''#searchResults > div:nth-child(1) > div'''

num_results = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, num_results_selector)))
time.sleep(1)
# Re-fetch the element
num_results = driver.find_element(By.CSS_SELECTOR, num_results_selector)
res = num_results.text.split(' ')
num_clicks = int(res[-1]) // int(res[-3]) + 1
print(f'There are {num_clicks} pages to click through.')

# Nested for loop to click through all search results
for click in range(num_clicks):

    for item in range(1, 21):
        try:
            elem = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="searchResults"]/table/tbody[{item}]/tr[1]/td[2]/a'))
            )
            
            # Scroll the element into view to ensure it's clickable
            driver.execute_script("arguments[0].scrollIntoView(true);", elem)
            time.sleep(2) 

            elem.click()

            # Switch to new window/tab
            driver.switch_to.window(driver.window_handles[2])
            time.sleep(2)  

            item_key = driver.current_url.split('/')[-1]
            item_key = item_key.replace('?', '_')

            res_dict[item_key] = driver.page_source

            with open(f'{item_key}.txt', encoding='utf-8', mode='w+') as file:
                file.write(driver.page_source)

            driver.close()

            # Switch back to the original window/tab
            driver.switch_to.window(driver.window_handles[1])

        except:
            continue  


    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) 

    # Click on next page once 20 results have been saved
    try:
        next_page = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchResults"]/div[3]/section/ul/li[3]/a/em'))
        )
    except:
        next_page = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchResults"]/div[3]/section/ul/li[1]/a/em'))
        )

    driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
    time.sleep(2)
    driver.execute_script("arguments[0].click();", next_page)
driver.quit()

Search parameters submitted.
There are 10 pages to click through.


In [6]:
# assert len(res_dict.keys()) == int(res[-1]), "It looks like not all the results were stored!"

In [7]:
os.chdir(curr)

In [8]:
def extract_content(html_string):
    soup = bs(html_string, 'html.parser')
    sitting_date = soup.find('td', string='Sitting Date:').find_next('td').span.get_text(strip=True)
    section_name = soup.find('td', string='Section Name:').find_next('td').span.get_text(strip=True)
    mps_speaking = soup.find('td', string='MPs Speaking:').find_next('td').span.get_text(strip=True)
    text_content = soup.select_one('div.hansardContent div.reportTable').get_text(strip=True)


    data_dict = {
        "Sitting Date": sitting_date,
        "Section Name": section_name,
        "MPs Speaking": mps_speaking,
        "Content": text_content
    }

    return data_dict

directory_path = f"./{output_directory}"

data_json = f"./{directory_path}/data.json"

# Create an empty list to store extracted content from all files
all_data = []

for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        with open(os.path.join(directory_path, file_name), 'r', encoding='utf-8') as txtfile:
            content = txtfile.read()
            data_dict = extract_content(content)
            all_data.append(data_dict)

# Save the list of extracted content to a JSON file
with open(data_json, 'w', encoding='utf-8') as jsonfile:
    json.dump(all_data, jsonfile, ensure_ascii=False, indent=4)



In [10]:

URL = "https://www.parliament.gov.sg/mps/list-of-current-mps"

def scrape_mp():
    response = requests.get(URL)
    
    if response.status_code != 200:
        print("Failed to retrieve webpage.")
        return []

    soup = bs(response.content, 'html.parser')
    
    # define selectors
    mp_data = {'Name':[], 'Constituency':[]}
    mp_container = soup.find('ul', class_='list')

    # find all 'li' elements inside the 'ul' container
    mp_items = mp_container.find_all('li') if mp_container else []
    
    for container in mp_items:
        name_element = container.find('a')
        constituency_element = container.find('div', class_="col-md-6 col-xs-11 mp-sort constituency")
        print(name_element.text.strip())

        if name_element and constituency_element:
            name = name_element.text.strip()
            constituency = constituency_element.text.strip()
            mp_data['Name'].append(name)
            
            mp_data['Constituency'].append(constituency)
    return mp_data

mp_data = scrape_mp()
# Define the path for the JSON file
filename = './scraped_data/mp.json'

# Write the dictionary to the file
with open(filename, 'w') as file:
    json.dump(mp_data, file)




Mr Ang Wei Neng
Mr Baey Yam Keng
Miss Cheryl Chan Wei Ling
Mr Chan Chun Sing
Ms Usha Chandradas
Mr Chee Hong Tat
Mr Edward Chia Bing Hui
Mr Chong Kee Hiong
Mr Desmond Choo
Mr Chua Kheng Wee Louis
Mr Eric Chua
Mr Keith Chua
Mr Darryl David
Mr Christopher de Souza
Ms Foo Mee Har
Ms Grace Fu Hai Yien
Mr Gan Kim Yong
Ms Gan Siow Huang
Mr Gan Thiam Poh
Mr Gerald Giam Yean Song
Mr Derrick Goh
Ms He Ting Ru
Mr Heng Chee How
Mr Heng Swee Keat
Mr Shawn Huang Wei Zhong
Ms Indranee Rajah
Mr S Iswaran
Dr Janil Puthucheary
Dr Amy Khor Lean Suan
Dr Koh Poh Koon
Mr Kwek Hian Chuan Henry
Mr Desmond Lee
Mr Lee Hsien Loong
Mr Mark Lee
Mr Leong Mun Wai
Mr Liang Eng Hwa
Mr Lim Biow Chuan
Assoc Prof Jamus Jerome Lim
Ms Sylvia Lim
Dr Lim Wee Kiak
Ms Low Yen Ling
Ms Mariam Jaafar
Mr Masagos Zulkifli Bin Masagos Mohamad
Dr Mohamad Maliki Bin Osman
Mr Mohd Fahmi Bin Aliman
Mr Muhamad Faisal Bin Abdul Manap
Assoc Prof Dr Muhammad Faishal Ibrahim
Mr Murali Pillai SC
Ms Nadia Ahmad Samdin
Dr Ng Eng Hen
Ms Ng Ling